In [1]:
import pandas as pd

In [2]:
df_all = pd.read_csv('error_analysis_df.tsv',sep='\t')
df_per_loc = pd.read_csv('error_analysis_df_per_loc.tsv',sep='\t')

In [16]:
def mention_boundaries(row): # error class
    pred_labels = [lab.split('-')[-1] for lab in eval(row['predicted_labels'])] # was a string, need a list
    true_labels = [lab.split('-')[-1] for lab in eval(row['true_labels'])]
    return pred_labels == true_labels

In [17]:
def false_positive(row): # error class
    pred_labels = eval(row['predicted_labels']) # was a string, need a list
    true_labels = eval(row['true_labels'])
    return pred_labels != true_labels and set(true_labels) == set('O')

In [19]:
def false_negative(row): # error class
    pred_labels = eval(row['predicted_labels']) # was a string, need a list
    true_labels = eval(row['true_labels'])
    return pred_labels != true_labels and set(pred_labels) == set('O')

In [ ]:
# for people vs. places, see df_per_loc

In [23]:
df_false_positive = pd.DataFrame.from_records([df_all.iloc[i] for i in range(len(df_all))
                                               if false_positive(df_all.iloc[i])]) # suboptimal but works

In [149]:
df_false_negative = pd.DataFrame.from_records([df_all.iloc[i] for i in range(len(df_all))
                                               if false_negative(df_all.iloc[i])]) # suboptimal but works

In [26]:
df_mention_boundaries = pd.DataFrame.from_records([df_all.iloc[i] for i in range(len(df_all))
                                               if mention_boundaries(df_all.iloc[i])]) # suboptimal but works

In [74]:
def print_row(row):
    print(row['model_name'], ',',
          row['test_set'].split('.')[0], ',',
          'sentence #',row['sentence_number'])
    print(row['left_context'][-140:]) # a bit shorter
    print('+')
    print(row['right_context'][:140])
    print('_____')
    print('mention, predicted, true')
    print(row['mention_tokens'], row['predicted_labels'],
         row['true_labels'])
    
    return row['mention_tokens'], row['model_name'], row['test_set'], row['sentence_number']

In [75]:
# df_mention_boundaries.iloc[267]

In [76]:
def analyse_row(ind, df=df_mention_boundaries):
    mention, model_name, test_set, sentence_number = print_row(df.iloc[ind])
    # looking for other models with the same mistake
    df_same_shit_bro = df[(df['test_set'] == test_set) &
                      (df['sentence_number'] == sentence_number)&
                      (df['model_name'] != model_name)&
                      (df['mention_tokens'] == mention)
                     ]
    print('Other models with the same mistake:')
    print(set(df_same_shit_bro['model_name']))

In [147]:
df_false_positive[df_false_positive['model_name'] == 'BERT-multi-cased'].sample(5)

,Unnamed: 0,model_name,test_set,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
1561,3766,BERT-multi-cased,ned.testb,35,""" Toch betekent de afstemming op de",arbeidsmarkt niet dat de algemene vorming uit ...,['arbeidsmarkt'],['B-LOC'],['O']
1961,4639,BERT-multi-cased,ned.testb,736,NaN,sectie,['sectie'],['B-LOC'],['O']
1435,3316,BERT-multi-cased,test_NHA.txt,18,1822 . Aug.s 21,. N=o 179 . Protest van Nonbetaling . Haarlem ...,['.'],['I-TIME'],['O']
1757,4133,BERT-multi-cased,ned.testb,339,"De gemengde intercommunales , die 70 procent v...",31 december gratis het signaal door te mogen s...,"['31', 'december']","['B-TIME', 'I-TIME']","['O', 'O']"
1613,3863,BERT-multi-cased,ned.testb,120,NaN,sectie,['sectie'],['B-LOC'],['O']


In [119]:
# df_mention_boundaries[df_mention_boundaries['true_labels'].str.find('LOC') != -1].sample(5)

,Unnamed: 0,model_name,test_set,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
622,4509,BERT-multi-cased,ned.testb,632,Olano (,Spa ) 1:09 -- 3 .,['Spa'],['I-LOC'],['B-LOC']
630,4906,BERT-multi-cased,ned.testb,983,Jeff wilde anders en beter worden dan zijn vad...,York een van Tims liedjes ten gehore had gebra...,['York'],['B-LOC'],['I-LOC']
468,3217,BERTje,ned.testb,646,Hier komt mijn kreet en pleidooi : laten wij a...,"Afrika , mee instappen in partenariaten en op ...",['Afrika'],['I-LOC'],['B-LOC']
331,1940,BERTje,test_NHA.txt,33,"nommer 3437 en 3440 geheel, samen groot omstre...",3159 geheel en een daaraan Zuidelijk grenzende...,['3159'],['I-LOC'],['B-LOC']
789,6257,WikiNEuRal,test_SA.txt,88,Verklaerende hij 2e: get: beneevens de 3e: 4e:...,Peperstraet . Al het welk voorsz: zij gets: vo...,['Peperstraet'],['I-LOC'],['B-LOC']


In [151]:
df_false_negative.sample(5)

,Unnamed: 0,model_name,test_set,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
2391,6983,WikiNEuRal,test_VOC.txt,81,157 . Van Cormandel den 15en April Ao 1722 ver...,"bimilipatnam 740: d=os aangeno� men, tegens de...",['bimilipatnam'],['O'],['B-LOC']
2015,6048,WikiNEuRal,test_SA.txt,57,vreeselijke wijze haer 1e get: zoo dat de 2e :...,26 Julij 1750 . (was Gett:) N. C. Muijlman quo...,"['26', 'Julij', '1750', '.']","['O', 'O', 'O', 'O']","['B-TIME', 'I-TIME', 'I-TIME', 'I-TIME']"
1566,4693,BERT-multi-cased,ned.testb,783,Het magazine van de zee brengt reportages over...,kruidenkust ' .,['kruidenkust'],['O'],['B-LOC']
1024,3286,BERT-multi-cased,test_NHA.txt,6,"uitgezonderd, volgens de wet, met uitdrukkelyk...",. Goed voor der tig Centen Crommelin Numero Tw...,['.'],['O'],['I-PER']
1624,4873,BERT-multi-cased,ned.testb,954,De zwarten zijn bang dat hun cultuur ten onder...,Midden-Amerika .,['Midden-Amerika'],['O'],['B-LOC']


In [185]:
all_wrong_preds['GysBERT']['ned.testb'][17]

(93,
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [180]:
all_wrong_preds['BERTje']['ned.testb'][13]

(93,
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [192]:
df_all[(df_all['sentence_number']==13)&(df_all['test_set']=='ned.testb')]

,Unnamed: 0,model_name,test_set,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
781,781,GysBERT,ned.testb,13,Vincent De Poortere is de vroegere baas van,LDP en eigenaar van Balsan .,['LDP'],['B-LOC'],['O']
782,782,GysBERT,ned.testb,13,Vincent De Poortere is de vroegere baas van LD...,Balsan .,['Balsan'],['B-LOC'],['O']
2438,2438,BERTje,ned.testb,13,-- Scenario voor opnieuw opstarten van failliete,Louis De Poortere,"['Louis', 'De', 'Poortere']","['B-PER', 'I-PER', 'I-PER']","['O', 'O', 'O']"
3736,3736,BERT-multi-cased,ned.testb,13,Vincent De Poortere is de vroegere baas van LD...,Balsan .,['Balsan'],['B-PER'],['O']
7173,7173,WikiNEuRal,ned.testb,13,NaN,Uefa en Fifa doen concessie inzake transfers,['Uefa'],['B-ORG'],['O']
7174,7174,WikiNEuRal,ned.testb,13,Uefa en,Fifa doen concessie inzake transfers,['Fifa'],['B-ORG'],['O']


In [190]:
analyse_row(789,df_all)

GysBERT , ned , sentence # 17
-- Scenario voor opnieuw opstarten van failliete
+
Louis De Poortere
_____
mention, predicted, true
['Louis', 'De', 'Poortere'] ['B-PER', 'I-PER', 'I-PER'] ['O', 'O', 'O']
Other models with the same mistake:
set()


In [90]:
import pickle
all_wrong_preds=pickle.load(open('all_wrong_preds.p','rb'))

In [155]:
df_conll = df_all[df_all['test_set'] == 'ned.testb']
df_conll_bertje = df_conll[df_conll['model_name']=='BERTje']
df_conll_bertje.sample(5)

,Unnamed: 0,model_name,test_set,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
2883,2883,BERTje,ned.testb,356,Lierse-Bordeaux op,12 september,"['12', 'september']","['B-TIME', 'I-TIME']","['O', 'O']"
3043,3043,BERTje,ned.testb,492,NaN,L'Hebdo RTBF 1 20.00,"[""L'Hebdo""]",['B-PER'],['O']
2967,2967,BERTje,ned.testb,421,Lokeren-trainer Georges Leekens reageerde opge...,Youla .,['Youla'],['B-LOC'],['B-PER']
2641,2641,BERTje,ned.testb,179,"vrijdag ,",1 september 2000,"['1', 'september']","['B-TIME', 'I-TIME']","['O', 'O']"
2769,2769,BERTje,ned.testb,291,"vrijdag ,",1 september 2000,"['1', 'september']","['B-TIME', 'I-TIME']","['O', 'O']"
